<a href="https://colab.research.google.com/github/Rotem2411/alephBERTgimmelDalet/blob/main/Sentiment_LDA1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import csv
import pandas as pd
import nltk
from collections import Counter
from string import punctuation
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
def read_csv_file(file_path):
    data = []
    with open(file_path, 'r', newline='', encoding='utf-8') as csvfile:
        csv_reader = csv.reader(csvfile)
        for row in csv_reader:
            data.append(row)
    return data

In [3]:
    file_path = 'sentiments.csv'
    df = pd.read_csv(file_path)
    print(df.head(10))

    # Total number of samples
    num_samples = len(df)
    print("Total number of samples:", num_samples)

    # Sentiment distribution
    Sentiment_distribution = {}
    for index, row in df.iterrows():
        Sentiment_label = row['tag']
        if Sentiment_label not in Sentiment_distribution:
            Sentiment_distribution[Sentiment_label] = 1
        else:
            Sentiment_distribution[Sentiment_label] += 1
    print("Sentiment Distribution:")
    for Sentiment_label, count in Sentiment_distribution.items():
        print(f"{Sentiment_label}: {count} tags")

   id                                               text tag category class  \
0   1                    האריות של הצל חזק פה בתגובות...   ש  ECONOMY    b1   
1   2  זמרת תעשייה רק מפרסומות מפורסמת\n  אבל אין קהל...   ש  ECONOMY    b1   
2   4  את לא יורקת לבאר שממנה שתית. יפה יפה, אבל חסרת...   ש  ECONOMY    b1   
3   5  שמעון\n  משתתף בפרסומת למילקי במקום לעורר מודע...   ש  ECONOMY    b1   
4   6  הצחקתם אותי\n  מה כל כך אמייזינג בחברה עם אפס ...   ש  ECONOMY    b1   
5   7  איסו חברת הייטק מצליחה צריכה פרסומות שיבואו לע...   ש  ECONOMY    b1   
6   8           לא מכיר אותה ולא שמעתי אפילו שיר אחד שלה   ש  ECONOMY    b1   
7   9  וואללה לא יודע מה מתלהבים מנגה ארז. זה בכלל לא...   ש  ECONOMY    b1   
8  10  אהובה שגיא\n  נגה גדולה.זמרת מעולה.פרסומת מדלי...   ח  ECONOMY    b1   
9  11  הפרסומת של אמדוקס נראית מאוד קודרת ואפורה, כמו...   ש  ECONOMY    b1   

   total_tags  selected_tag  polarity  
0           2             2       1.0  
1           2             2       1.0  
2         

In [4]:
features = df['text']
labels = df['tag']

X_train, X_test, y_train, y_test = train_test_split(features,labels,test_size = 0.2, random_state=42)
#X_train, X_test, y_train, y_test = train_test_split(X_train,y_train,test_size = 0.5, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_test,y_test,test_size = 0.5, random_state=42)

print("Data distribution:\n- Train: {} \n- Validation: {} \n- Test: {}".format(len(y_train),len(y_val),len(y_test)))

Data distribution:
- Train: 60120 
- Validation: 7515 
- Test: 7516


In [5]:
def cleanwords(text):
    wn = nltk.WordNetLemmatizer()
    stopword = nltk.corpus.stopwords.words('hebrew')
    tokens = nltk.word_tokenize(text)
    lower = [word.lower() for word in tokens]
    no_stopwords = [word for word in lower if word not in stopword]
    no_alpha = [word for word in no_stopwords if word.isalpha()]
    lemm_text = [wn.lemmatize(word) for word in no_alpha]
    clean_text = lemm_text
    return clean_text

In [6]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
tfidf_vect = TfidfVectorizer(analyzer=cleanwords)
tfidf_vect_fit=tfidf_vect.fit(X_train)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [7]:
def vectorize(data, tfidf_vect_fit):
    X_tfidf = tfidf_vect_fit.transform(data)
    words = tfidf_vect_fit.get_feature_names_out()
    X_tfidf_df = pd.DataFrame(X_tfidf.toarray())
    X_tfidf_df.columns = words
    return(X_tfidf_df)

In [8]:
X_train = vectorize(X_train, tfidf_vect_fit)

LDA - Latent Dirichlet allocation

In [9]:
tfidf_matrix = tfidf_vect.fit_transform(X_train)
num_topics = 3
lda_model = LatentDirichletAllocation(n_components=num_topics, random_state=100)
lda_matrix = lda_model.fit_transform(tfidf_matrix)

In [10]:
# Print the top words for each topic
feature_names = tfidf_vect.get_feature_names_out()
for topic_idx, topic in enumerate(lda_model.components_):
    top_words = [feature_names[i] for i in topic.argsort()[:-11:-1]]
    print(f"Topic {topic_idx + 1}: {', '.join(top_words)}")

Topic 1: עויין, שבחירות, ואופא, שמחקו, עורכת, קצורי, ללמידה, מאפילה, קוק, ומסתיים
Topic 2: חינמית, ובמשבר, יורדות, צברתי, החזיק, הטכנולוגייה, מנובלים, התייעלו, מקדשת, כלום
Topic 3: מיעוטם, במעטפות, משתמשיים, לאחד, שמספר, שמיובאים, מתחסנות, בטיחותית, הסמטלנים, והחברים


In [11]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt
# Generate the word cloud.
text = str(feature_names)
text = cleanwords(text)
wordcloud = WordCloud().generate(text)

# Display the word cloud.
plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation="bilinear")
plt.axis("off")
plt.title(f"Top words for {topic_idx}", fontsize=16)
plt.show()

TypeError: expected string or bytes-like object

In [12]:
text = str(feature_names)
text

"['a' 'about' 'accidentally' ... 'ײסודה' 'בּטוח' 'בּמקצועם']"

In [13]:
text = cleanwords(text)
text

['a']